In [30]:
% matplotlib inline

import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
H1_data = np.genfromtxt('H1_times.csv')
L1_data = np.genfromtxt('L1_times.csv')

In [3]:
H1_cbc = H1_data[np.where(H1_data[:,9] == 1)]
H1_burst = H1_data[np.where(H1_data[:,10] == 1)]
H1_stoch = H1_data[np.where(H1_data[:,11] == 1)]

print(H1_cbc.shape)

(16, 12)


In [16]:
H1_terr = np.array([])
for event in H1_data:
    b = event[0]
    e = event[1]
    
    # find H1 events whose beginings and ends do not overlap with L1 events
    b_novr = len(np.where((b > L1_data[:,0]) & (b < L1_data[:,1]))[0]) == 0
    e_novr = len(np.where((e > L1_data[:,0]) & (e < L1_data[:,1]))[0]) == 0
    
    if (b_novr and e_novr):
        if (H1_terr.size == 0):
            H1_terr = np.array(event)
        else:
            H1_terr = np.vstack((H1_terr, event))

In [24]:
training_events = np.concatenate((H1_cbc,H1_burst,H1_stoch,H1_terr))
train = np.delete(training_events,[0,1,7,8,9,10,11], axis=1)

train_labels = np.concatenate((np.full(len(H1_cbc), 0),\
                               np.full(len(H1_burst), 1),\
                               np.full(len(H1_stoch), 2),\
                               np.full(len(H1_terr), 3)))

/home/jacaseyclyde/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(5, 1) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/jacaseyclyde/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(0, 2) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)
/home/jacaseyclyde/anaconda2/lib/python2.7/site-packages/numpy/core/numeric.py:301: FutureWarning: in the future, full(429, 3) will return an array of dtype('int64')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


In [25]:
print(train.shape)
print(train_labels.shape)

(450, 5)
(450,)


In [29]:
# scale data for mlp
scaler = StandardScaler()  

scaler.fit(train)  
train_scale = scaler.transform(train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
mlp = MLPClassifier()
parameters = {'hidden_layer_sizes':[(3,)],'activation':['logistic'],'solver':['lbfgs'],'alpha':(10.0 ** -np.arange(1, 7))}
clf = GridSearchCV(mlp, parameters)
clf.fit(train,train_labels)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').